In [ ]:
import re
from pathlib import Path

import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")

In [ ]:
dfs = []
for fn in snakemake.input["results"]:
    df = pd.read_csv(fn, delimiter=";")
    df["csp"] = "CSP" if "_with_csp" in fn else "no CSP"

    dfs.append(df)

df = pd.concat(dfs)

In [ ]:
df = df[df["subcategory"].str.startswith("Total installed capacity ")]

df["technology"] = df["subcategory"].str.replace("Total installed capacity ", "")

rename_technologies = {
    "csp-tower": "CSP",
    "offwind": "wind offshore",
    "onwind": "wind onshore",
    "solar-utility": "PV",
}

for old_name, new_name in rename_technologies.items():
    df["technology"] = df["technology"].str.replace(old_name, new_name)

df["value"] /= 1e3  # generation capacity in GW instead of MW

# Consider efficiency of power block for csp-tower
# resulting number is ficitious (does not consider power block capacity)
# but allows for comparison of magnitudes vs. MWp of PV and wind
df.loc[df["technology"] == "CSP", "value"] *= 0.412

In [ ]:
for esc in ["hvdc-to-elec", "pipeline-h2-to-elec"]:
    for exporter in ["MA", "TN", "SA"]:
        elec = df.query("esc == @esc and exporter == @exporter").reset_index(drop=True)

        fig, axes = plt.subplots(
            nrows=1, ncols=elec["scenario"].unique().shape[0], sharey=True
        )

        # Axes <-> scenario mapping: Index where each scenario is plotted in figure
        scenario_order = {
            "unbuffered": 0,
            "daily": 1,
            "weekly": 2,
            "biweekly": 3,
            "monthly": 4,
            "quaterly": 5,
            "annually": 6,
        }

        for name, group in elec.groupby("scenario"):
            group = group[["csp", "technology", "value"]].pivot_table(
                index="csp", columns="technology", values="value"
            )

            ax = axes[scenario_order[name]]
            group.plot.bar(stacked=True, rot=90, ax=ax, legend=0)

            ax.set_ylabel("Installed capacity [GW]")
            ax.set_xlabel(name)

        # Single legend for all subplots
        handles, labels = ax.get_legend_handles_labels()
        legend = fig.legend(
            handles, labels, loc="center right", bbox_to_anchor=(1.2, 0.5)
        )
        fig.suptitle(f"{exporter}: {esc}")
        # fig.tight_layout()

        fig.savefig(
            [fn for fn in snakemake.output["figures"] if exporter in fn and esc in fn][
                0
            ],
            dpi=300,
            bbox_inches="tight",
            bbox_extra_artists=(legend,),
        )